In [9]:
import cv2
##计算卷积
import numpy as np
import skimage.data

#测试数据
source_map = np.array(list('1110001110001110011001100')).astype(np.int32)
source_map = source_map.reshape(5,5)
print(f'原始数据:\n{source_map}')

#滤波器(Filter)
filter1 = np.array(list('101010101')).astype(np.int32).reshape(3,3)
print(f'滤波器1:\n{filter1}')

#计算卷积
##初始化计算结果的矩阵
width = height = source_map.shape[0] - filter1.shape[0] + 1
result = np.zeros((width, height))

# 计算每一格
for i in range(width):
    for j in range(height):
        # 获取当前格子对应的数据
        value = source_map[i:i+filter1.shape[0], j:j+filter1.shape[1]] * filter1
        # 计算
        result[i, j] = np.sum(value)
print(f'卷积结果:\n{result}')


原始数据:
[[1 1 1 0 0]
 [0 1 1 1 0]
 [0 0 1 1 1]
 [0 0 1 1 0]
 [0 1 1 0 0]]
滤波器1:
[[1 0 1]
 [0 1 0]
 [1 0 1]]
卷积结果:
[[4. 3. 4.]
 [2. 4. 3.]
 [2. 3. 4.]]


In [13]:
'''
    使用scipy的卷积函数验证

    二维卷积Conv2D一般用来图像上面;一维卷积Conv1D可用于语音或者文字方面,只考虑上下文;三维卷积Conv3D可用于
    立体的对象
'''
from scipy.signal import convolve2d
#计算二维卷积
convolve2d(source_map, filter1,mode='valid')

array([[4, 3, 4],
       [2, 4, 3],
       [2, 3, 4]], dtype=int32)

In [ ]:
'''
    各式卷积
   pip install scikit-image
   pip install opencv-python
'''
#1.定义一个卷积的影像转换函数
from skimage.exposure import rescale_intensity
import skimage
import cv2


def convolve(image,kernel):
    #   取得图像和滤波器的宽高
    (iH,iW) = image.shape[:2]
    (kH,kW) = kernel.shape[:2]

    # 计算padding = 'same'单边所需的补零行数
    pad = int((kW-1)/2)
    image = cv2.copyMakeBorder(image,pad,pad,pad,pad,cv2.BORDER_REPLICATE)
    output = np.zeros((iH,iW),dtype='float32')

    #卷积
    for y in np.arange(pad,iH+pad):
        for x in np.arange(pad,iW+pad):
            roi = image[y-pad:y+pad+1,x-pad:x+pad+1] #裁切图像
            k = (roi *kernel).sum() #卷积计算
            output[y-pad, x-pad] = k #更新计算结果的矩阵

    #调整影响色彩深浅范围至(0,255)
    output = rescale_intensity(output,in_range=(0,255))
    output = (output*255).astype('uint8')
    #回传结果影像
    return output

#获取skimage内置的图像
image = skimage.data.chelsea()
cv2.imshow('Original',image)

#2.灰阶化
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
cv2.imshow('Gray',gray)

#绑定Enter键关闭窗口
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
## 3.模糊化
###小模糊filter
import numpy as np

smallBlur = np.ones((7,7),dtype='float') * (1.0/(7*7))

###卷积
convolveOutput = convolve(gray,smallBlur)
opencvOutput = cv2.filter2D(gray,-1,smallBlur)
cv2.imshow('little Blur',convolveOutput)

### 大模糊
largeBlur = np.ones((21,21),dtype='float') * (1.0/(21*21))

### 卷积
convolveOutput = convolve(gray,largeBlur)
opencvOutput = cv2.filter2D(gray,-1,largeBlur)
cv2.imshow('large Blur',convolveOutput)

#按Enter键关闭窗口
cv2.waitKey(0)
cv2.destroyAllWindows()


In [7]:
## 4.锐化(Sharpen):可使图像的对比更加明显
import numpy as np
sharpen = np.array((
    [0,-1,0],
    [-1,5,-1],
    [0,-1,0]
),dtype='int')

### 卷积
convolveOutput = convolve(gray,sharpen)
opencvOutput = cv2.filter2D(gray,-1,sharpen)
cv2.imshow('Sharpen',convolveOutput)

##按Enter键关闭窗口
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
## 5.Laplacian边缘检测
laplacian = np.array((
    [0,1,0],
    [1,-4,1],
    [0,1,0]
),dtype='int')

###卷积
convolveOutput = convolve(gray,laplacian)
opencvOutput = cv2.filter2D(gray,-1,laplacian)
cv2.imshow('Laplacian edge detection',convolveOutput)

### Enter关闭窗口
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
## 6.Sobel X轴边缘检测:沿着X轴检测边缘,可以检测垂直线特征
sobelX = np.array((
    [-1,0,1],
    [-2,0,2],
    [-1,0,1]
),dtype='int')

###卷积
convolveOutput = convolve(gray,sobelX)
opencvOutput = cv2.filter2D(gray,-1,sobelX)
cv2.imshow('Sobel X',convolveOutput)

### Enter关闭窗口
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:

## 7.Sobel Y轴边缘检测:沿着Y轴检测边缘,可以检测水平线特征
sobelY = np.array((
    [-1,-2,-1],
    [0,0,0],
    [1,2,1]
),dtype='int')

###卷积
convolveOutput = convolve(gray,sobelY)
opencvOutput = cv2.filter2D(gray,-1,sobelY)
cv2.imshow('Sobel Y',convolveOutput)

## Enter关闭窗口
cv2.waitKey(0)
cv2.destroyAllWindows()